In [1]:
import time

from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoProcessor, AutoModelForCausalLM
from googletrans import Translator
import os
import torch

In [2]:
translator = Translator()
def translate_text(text, dest_lang='ru', src='en'):
    translation = translator.translate(text, dest=dest_lang, src=src)
    return translation.text

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
processor_git = AutoProcessor.from_pretrained("microsoft/git-base-coco")
model_git = AutoModelForCausalLM.from_pretrained("microsoft/git-base-coco").to(device)

In [5]:
def generate_caption_git(image):
    pixel_values = processor_git(images=image, text='a museum photography of', return_tensors="pt").pixel_values.to(device)

    generated_ids = model_git.generate(pixel_values=pixel_values, max_length=50)
    generated_caption = processor_git.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return translate_text(generated_caption)

In [6]:
processor_blip = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model_blip = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)

In [7]:
def generate_desc(image):
    # conditional image captioning
    text = "a museum photography of"
    inputs = processor_blip(image, text, return_tensors="pt").to(device)

    out = model_blip.generate(**inputs)
    answer = processor_blip.decode(out[0], skip_special_tokens=True)
    return translate_text(answer)
    #return processor.decode(out[0], skip_special_tokens=True)

In [13]:
import time

flag = True
start_time = None
end_time = None

for img in os.listdir("../data/yolo_best/val/Редкие книги")[:101]:
    if flag:
        start_time = time.time()
        flag = False
    img = Image.open(os.path.join("../data/yolo_best/val/Редкие книги", img))
    generate_caption_git(img)
end_time = time.time()
print("Time: ", end_time - start_time)

Time:  45.8468177318573


In [8]:
import time

flag = True
start_time = None
end_time = None

for img in os.listdir("../../data/yolo_best/val/Редкие книги")[:101]:
    if flag:
        start_time = time.time()
        flag = False
    img = Image.open(os.path.join("../../data/yolo_best/val/Редкие книги", img))
    generate_caption_git(img)
end_time = time.time()
print("Time: ", end_time - start_time)

Time:  43.33316206932068
